In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import GRU
from keras.datasets import imdb
import numpy as np
import pandas as pd
import os

Using TensorFlow backend.


In [3]:
names = ['~', 'stageX', 'stageY', 'headX', 'headY' , 'commandX', 'commandY']
df = pd.read_csv('C:\\Users\\Felix\\Desktop\\毕设\\数据\\XYdata-13600.txt',sep=',',header=None,names=names,na_values=['-1'], index_col=False)

print("Starting file:")
print(df[0:10])

print("Ending file:")
print(df[-10:])

Starting file:
   ~   stageX   stageY    headX    headY  commandX  commandY
0  1  4.92227  2.44792  210.329  195.888  0.008663 -0.002653
1  1  4.91331  2.44404  209.282  194.188  0.010410 -0.001959
2  1  4.91337  2.44187  209.423  189.576  0.013621 -0.001917
3  1  4.92063  2.44000  209.246  189.475  0.014675 -0.001758
4  1  4.93597  2.43820  208.985  191.086  0.014294 -0.001522
5  1  4.95664  2.43661  209.029  191.863  0.013883 -0.001454
6  1  4.97924  2.43573  208.586  193.370  0.012949 -0.001140
7  1  5.00523  2.43429  208.995  196.032  0.011188 -0.001224
8  1  5.03304  2.43332  209.020  198.801  0.009119 -0.001181
9  1  5.05857  2.43253  208.622  200.209  0.007674 -0.000918
Ending file:
       ~   stageX   stageY    headX    headY  commandX  commandY
11228  1 -2.46791  4.14121  224.038  182.363  0.021421 -0.014366
11229  1 -2.44529  4.12766  223.888  181.865  0.021651 -0.014366
11230  1 -2.42037  4.11245  223.555  180.099  0.022690 -0.014233
11231  1 -2.39785  4.09749  224.016  176.

In [4]:
freq=300#伺服控制频率
a11=0.0001777326832096361
a12=0.005308842744342313
a21=0.00534395497810797
a22=-0.00019522174569135787
centerX=208
centerY=208

In [7]:
#转浮点
df['headX'] = df['headX'].astype(float)
df['stageX'] = df['stageX'].astype(float)
df['stageY'] = df['stageY'].astype(float)
df['headY'] = df['headY'].astype(float)
df['commandX'] = df['commandX'].astype(float)
df['commandY'] = df['commandY'].astype(float)

#赋值
headX = df['headX']
stageX = df['stageX']
headY = df['headY']
stageY = df['stageY']
commandX = df['commandX']
commandY = df['commandY']
#算坐标stagetarget和速度headspeed
headErrorX=centerX-headX
headErrorY=centerY-headY
stageErrorX=headErrorX * a11 + a12 * headErrorY
stageErrorY=headErrorX * a21 + a22 * headErrorY
stageTargetX=(stageX+stageErrorX).tolist()
stageTargetY=(stageY+stageErrorY).tolist()
headSpeedX=(np.diff(stageTargetX)*300).tolist()
headSpeedY=(np.diff(stageTargetY)*300).tolist()

In [8]:
def to_sequences(seq_size, obs):
    x = []
    y = []

    for i in range(len(obs)-SEQUENCE_SIZE):
        #print(i)
        window = obs[i:(i+SEQUENCE_SIZE)]
        after_window = obs[i+SEQUENCE_SIZE]
        window = [x for x in window]
        #print("{} - {}".format(window,after_window))
        x.append(window)
        y.append(after_window)
        
    return np.array(x),np.array(y)

In [9]:
#batch size
SEQUENCE_SIZE = 10
#算训练和预测的xy
x_train,x_pred = to_sequences(SEQUENCE_SIZE,stageTargetX)
y_train,y_pred = to_sequences(SEQUENCE_SIZE,stageTargetY)
xspeed_train,xspeed_pred = to_sequences(SEQUENCE_SIZE,headSpeedX)
yspeed_train,yspeed_pred = to_sequences(SEQUENCE_SIZE,headSpeedY)

In [10]:
#删去多余维度
x_train = np.delete(x_train,0,0)
y_train = np.delete(y_train,0,0)
x_pred = np.delete(x_pred,0,0)
y_pred = np.delete(y_pred,0,0)
#换行列
combine = np.array([x_train,y_train,xspeed_train,yspeed_train])
combine_pred = np.array([x_pred,y_pred])
combine = combine.swapaxes(0,1)
combine_pred = combine_pred.swapaxes(0,1)
combine = combine.swapaxes(1,2)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

print('Build model...')
model = Sequential()
model.add(GRU(32, dropout=0.0, input_shape=(None,4), recurrent_dropout=0.0,))
model.add(Dense(64))
model.add(Dense(64))
model.add(Dense(64))
model.add(Dense(2))
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True)
print('Train...')

model.fit(combine,combine_pred,callbacks=[monitor],verbose=2,epochs=1000)

Build model...




Train...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/1000





 - 3s - loss: 0.4197
Epoch 2/1000
 - 2s - loss: 0.0281
Epoch 3/1000
 - 2s - loss: 0.0129
Epoch 4/1000
 - 2s - loss: 0.0102
Epoch 5/1000
 - 2s - loss: 0.0068
Epoch 6/1000
 - 2s - loss: 0.0056
Epoch 7/1000
 - 2s - loss: 0.0062
Epoch 8/1000
 - 2s - loss: 0.0087
Epoch 9/1000
 - 2s - loss: 0.0049
Epoch 10/1000
 - 2s - loss: 0.0063
Epoch 11/1000
 - 2s - loss: 0.0048
Epoch 12/1000
 - 2s - loss: 0.0056
Epoch 13/1000
 - 2s - loss: 0.0055
Epoch 14/1000
 - 2s - loss: 0.0040
Epoch 15/1000
 - 2s - loss: 0.0032
Epoch 16/1000
 - 2s - loss: 0.0031
Epoch 17/1000
 - 3s - loss: 0.0053
Epoch 18/1000
 - 2s - loss: 0.0051
Epoch 19/1000
 - 2s - loss: 0.0054
Epoch 20/1000
 - 2s - loss: 0.0031
Epoch 21/1000
 - 3s - loss: 0.0027
Epoch 22/1000
 - 2s - loss: 0.0031
Epoch 23/1000
 - 3s - loss: 0.0018
Epoch 24/1000
 - 3s - loss: 0.0023
Epoch 25/1000
 - 2s - loss: 0.0044
Epoc

In [ ]:
names = ['~', 'stageX', 'stageY', 'headX', 'headY' , 'commandX', 'commandY']
df_test = pd.read_csv('C:\\Users\\Felix\\Desktop\\毕设\\数据\\XYdata-test.txt',sep=',',header=None,names=names,na_values=['-1'], index_col=False)

print("Starting file:")
print(df_test[0:10])

print("Ending file:")
print(df_test[-10:])

In [ ]:
df_test['headX'] = df_test['headX'].astype(float)
df_test['stageX'] = df_test['stageX'].astype(float)
df_test['stageY'] = df_test['stageY'].astype(float)
df_test['headY'] = df_test['headY'].astype(float)
df_test['commandX'] = df_test['commandX'].astype(float)
df_test['commandY'] = df_test['commandY'].astype(float)

In [ ]:
headX1 = df_test['headX']
stageX1 = df_test['stageX']
headY1 = df_test['headY']
stageY1 = df_test['stageY']
commandX1 = df_test['commandX']
commandY1 = df_test['commandY']
headErrorX1=centerX-headX1
headErrorY1=centerY-headY1
stageErrorX1=headErrorX1 * a11 + a12 * headErrorY1
stageErrorY1=headErrorX1 * a21 + a22 * headErrorY1
stageTargetX1=stageX1+stageErrorX1
stageTargetY1=stageY1+stageErrorY1
headSpeedX1=np.diff(stageTargetX1)*300
headSpeedY1=np.diff(stageTargetY1)*300

In [ ]:
SEQUENCE_SIZE = 100
x_test,x_testpred = to_sequences(SEQUENCE_SIZE,stageTargetX1)
y_test,y_testpred = to_sequences(SEQUENCE_SIZE,stageTargetY1)
xspeed_test,xspeed_test_pred = to_sequences(SEQUENCE_SIZE,headSpeedX1)
yspeed_test,yspeed_test_pred = to_sequences(SEQUENCE_SIZE,headSpeedY1)

In [ ]:
x_test = np.delete(x_test,0,0)
y_test = np.delete(y_test,0,0)
x_testpred = np.delete(x_testpred,0,0)
y_testpred = np.delete(y_testpred,0,0)

In [ ]:
combine_test = np.array([x_test,y_test,xspeed_test,yspeed_test])
combine_pred_test = np.array([x_testpred,y_testpred])
combine_test = combine_test.swapaxes(0,1)
combine_pred_test = combine_pred_test.swapaxes(0,1)
combine_test = combine_test.swapaxes(2,1)

In [ ]:
from sklearn import metrics

pred = model.predict(combine)
score = np.sqrt(metrics.mean_squared_error(pred[:,1],combine_pred[:,1]))
print("Score (RMSE): {}".format(score))